## Dogtor dog's temporal <big style="color: red;">3D</big> COVID19 visualization solution

<img src="https://s14-eu5.startpage.com/cgi-bin/serveimage?url=https:%2F%2Fsteamuserimages-a.akamaihd.net%2Fugc%2F272841093784221615%2F948967DA5537466A9509113E12C237641EB1506D%2F&sp=9ce3481ad1247d4f63b90d00de600702" alt="The Man himself" style="width:250px;height:200px;">
<audio autoplay>
  <source src="https://a.tumblr.com/tumblr_m9cqm4IkM31qkv3fno1.mp3" type="audio/mpeg"></audio>

This is a 2D visualization solution for temporal COVID19 epidemiological data available from: <strike><a href="https://www.acdc.com/">acdc</a></strike>, <a href="https://www.ecdc.europa.eu/en/publications-data/download-todays-data-geographic-distribution-covid-19-cases-worldwide">ecdc</a>.

In [1]:
from __future__ import print_function
import ipywidgets as widgets
import pandas as pd
import numpy as np
from sklearn import linear_model
from ipywidgets import interact, interactive, fixed, interact_manual
import plotly.graph_objects as go
from IPython.display import display
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
%matplotlib inline

#fetching the data
while True:
    try:
        df = pd.read_csv("https://opendata.ecdc.europa.eu/covid19/casedistribution/csv") 
        # below is a link to an old source
        #("https://www.ecdc.europa.eu/sites/default/files/documents/COVID-19-geographic-disbtribution-worldwide-"+ str(date.today())[:8] + str(int(str(date.today())[8:])-1) + ".xlsx")
        break
    except:
        print("Error fetching data")
        break

# function for fetching the requested countrydata
def preprocess(df, countrylist, days):

    dflist = []
    for country in countrylist:
        dfcountry = df[["dateRep","cases", "deaths", "popData2019"]][df["countriesAndTerritories"] == country]
        dfcountry['country'] = country
        dfcountry = dfcountry.iloc[::-1]
        dfcountry["cumCases"] = dfcountry["cases"].cumsum(axis = 0, skipna = True)
        dfcountry["cumDeaths"] = dfcountry["deaths"].cumsum(axis = 0, skipna = True)
        dfcountry["runDate"] = list(range(len(dfcountry)))
        dflist.append(dfcountry)
    dfTemporal = pd.concat(dflist)
    dfTemporal["dateRep"] = dfTemporal["dateRep"].astype("str")
    return(dfTemporal)

# function for plotting cases and deaths
def lineplotCaseDeath(df, countrylist, days, relPop, lm):

    figCases = go.Figure()
    dflm = pd.DataFrame(columns=["runDate","cumCases", "cumDeaths", "popData2019"])
    
    for country in countrylist:
        
        if relPop is False:
            z = df["cumCases"][df["country"] == country].tail(days)
        else:
            z = (df["cumCases"]/df["popData2019"])[df["country"] == country].tail(days)
            
        if lm is False:
            runDate = df["dateRep"].tail(days)
        else:
            runDate = df["runDate"].tail(days)
            dflm = dflm.append(df[["runDate","cumCases", "cumDeaths", "popData2019"]][df["country"] == country].tail(days), sort=False)
            
        figCases.add_trace(go.Scatter3d(x=runDate, y=df["popData2019"][df["country"] == country].tail(days),
                                      z=z,
                            mode='lines',
                            name=country + ' cases'))
    
    if lm is True:
        
        X = dflm[['runDate', 'popData2019']].values.reshape(-1,2)
        if relPop is False:
            Y = dflm['cumCases']
        else:
            Y = dflm['cumCases']/dflm['popData2019']

        ######################## Prepare model data point for visualization ###############################

        x = X[:, 0]
        y = X[:, 1]
        z = Y

        x_pred = np.linspace(min(dflm.runDate), max(dflm.runDate), days)   # range of date values
        y_pred = np.linspace(min(dflm.popData2019), max(dflm.popData2019), days)  # range of popData2019 values
        xx_pred, yy_pred = np.meshgrid(x_pred, y_pred)
        model_viz = np.array([xx_pred.flatten(), yy_pred.flatten()]).T

        ################################################ Train #############################################

        ols = linear_model.LinearRegression()
        model = ols.fit(X, Y)
        predicted = model.predict(model_viz)

        ############################################## Evaluate ############################################

        r2 = model.score(X, Y)
        
        ################################################ Plot ##############################################
        
        #figCases.add_trace(go.Scatter3d(x=xx_pred.flatten(), y=yy_pred.flatten(), z=predicted, mode='markers', name='linear model'))
        figCases.add_trace({'type': 'mesh3d',
                    'x': xx_pred.flatten(),
                    'y': yy_pred.flatten(),
                    'z': predicted,
                    'delaunayaxis':'x',
                    'color': 'green',
                    'opacity': 0.5
                   })
    

    figCases.update_layout(title="Cases:" + str(list(df["country"].unique())))
    
    figCases.show()
    
    figDeaths = go.Figure()
    
    for country in countrylist:
        
        df[df["country"] == country].tail(days)
        
        if relPop is False:
            z = df["cumDeaths"][df["country"] == country].tail(days)
        else:
            z = (df["cumDeaths"]/df["popData2019"])[df["country"] == country].tail(days)
        
        figDeaths.add_trace(go.Scatter3d(x=runDate, y=df["popData2019"][df["country"] == country].tail(days),
                                       z=z,
                            mode='lines',
                            name=country + ' deaths'))
        
    if lm is True:

        #dflm = df[["runDate","cumDeaths", "popData2019"]]
        
        X = dflm[['runDate', 'popData2019']].values.reshape(-1,2)
        if relPop is False:
            Y = dflm['cumDeaths']
        else:
            Y = dflm['cumDeaths']/dflm['popData2019']

        ######################## Prepare model data point for visualization ###############################

        x = X[:, 0]
        y = X[:, 1]
        z = Y

        x_pred = np.linspace(min(dflm.runDate), max(dflm.runDate), days)   # range of date values
        y_pred = np.linspace(min(dflm.popData2019), max(dflm.popData2019), days)  # range of popData2019 values
        xx_pred, yy_pred = np.meshgrid(x_pred, y_pred)
        model_viz = np.array([xx_pred.flatten(), yy_pred.flatten()]).T

        ################################################ Train #############################################

        ols = linear_model.LinearRegression()
        model = ols.fit(X, Y)
        predicted = model.predict(model_viz)

        ############################################## Evaluate ############################################

        r2 = model.score(X, Y)
        
        ################################################ Plot ##############################################
        
        #figDeaths.add_trace(go.Scatter3d(x=xx_pred.flatten(), y=yy_pred.flatten(), z=predicted, mode='markers', name='linear model'))
        figDeaths.add_trace({'type': 'mesh3d',
                            'x': xx_pred.flatten(),
                            'y': yy_pred.flatten(),
                            'z': predicted,
                            'delaunayaxis':'x',
                            'color': 'green',
                            'opacity': 0.5
                           })
    
    figDeaths.update_layout(title="Deaths:" + str(list(df["country"].unique())))
    figDeaths.show()

countrylist = []

def plotti(days, countries, Relation_To_Population, linear_model):
    global countrylist
    countrylist.append(countries)
    countrylist = list(set(countrylist))
    print(countrylist)
    df2 = preprocess(df, countrylist, days)
    return lineplotCaseDeath(df2, countrylist, days, Relation_To_Population, linear_model)

def resetCountries(b):
    global countrylist
    countrylist = []
    return countrylist

button = widgets.Button(description="reset countries")

button.on_click(resetCountries)

display(button)

interact_manual(plotti, days = widgets.IntSlider(min=10,max=300,value=30), 
                countries = widgets.Dropdown(options = df["countriesAndTerritories"].unique()), 
                Relation_To_Population=False, linear_model=False)


Button(description='reset countries', style=ButtonStyle())

interactive(children=(IntSlider(value=30, description='days', max=300, min=10), Dropdown(description='countrie…

<function __main__.plotti(days, countries, Relation_To_Population, linear_model)>

In [ ]:
figCases.write_html("coronaMultiLinearmodel.html")